<a href="https://colab.research.google.com/github/martin-fabbri/kaggle-competitions/blob/main/cdp-unlocking-climate-solutions/1_kaggle_cdp_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

Install Basemap

In [1]:
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 73.1 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgeos-dev amd64 3.6.2-1build2 [73.1 kB]
Fetched 73.1 kB in 1s (113 kB/s)
Selecting previously unselected package libgeos-dev.
(Reading database ... 144617 files and directories currently installed.)
Preparing to unpack .../libgeos-dev_3.6.2-1build2_amd64.deb ...
Unpacking libgeos-dev (3.6.2-1build2) ...
Setting up libgeos-dev (3.6.2-1bu

Install geopandas

In [1]:
!pip install --upgrade geopandas

     |████████████████████████████████| 972kB 4.0MB/s 
     |████████████████████████████████| 14.8MB 260kB/s 


In [3]:
# data processing libs
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json

# plotting libs
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

# set inline plotly
from plotly.offline import init_notebook_mode;
init_notebook_mode(connected=True)

# geospatial libs
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon
import geopandas as gpd
import folium
print(f'geopandas: {gpd.__version__}')


geopandas: 0.8.1


## Data

### Import Data

In [4]:
!pip install --upgrade --force-reinstall --no-deps kaggle
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p ~/.kaggle
!cp /content/gdrive/My\ Drive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c cdp-unlocking-climate-solutions
!ls -l /content
!unzip -o /content/cdp-unlocking-climate-solutions.zip

     |████████████████████████████████| 61kB 2.5MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=9c7987886763d3891ffe32a05dd6da4450fb06a3a3d5cf9823e0fa666f0e520e
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
  Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
Mounted at /content/gdrive
 98% 241M/245M [00:05<00:00, 37.7MB/s]
100% 245M/245M [00:05<00:00, 43.0MB/s]
total 251116
-rw-r--r-- 1 root root 257130814 Oct 15 03:36 cdp-unlocking-climate-solutions.zip
drwx------ 5 root root      4096 Oct 15 03:36 gdrive
drwxr-xr-x 1 root root      4096 Oct  5 16:31 sample_data
Archive:  /content/cdp-unlocking-climate-solutions.zip
  inflating: Cities/Cities Disclosing/2018_Cities_Disclosing_to_CDP.csv  
  inflating: Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv  
  inflating: Cities/Cities Di

In [6]:
# import corporate response data
cc_df = pd.read_csv('/content/Corporations/Corporations Responses/Climate Change/2019_Full_Climate_Change_Dataset.csv')
ws_df = pd.read_csv('/content/Corporations/Corporations Responses/Water Security/2019_Full_Water_Security_Dataset.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.



In [7]:
# import cities response df
cities_df = pd.read_csv("/content/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

In [8]:
# external data - import CDC social vulnerability index data - census
svi_df = pd.read_csv("/content/Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US.csv")

In [10]:
# cities metadata - lat,lon locations for US cities
cities_meta_df = pd.read_csv("/content/Supplementary Data/Simple Maps US Cities Data/uscities.csv")

# cities metadata - CDP metadata on organisation HQ cities
cities_cdpmeta_df = pd.read_csv("/content/Supplementary Data/Locations of Corporations/NA_HQ_public_data.csv")

### Helpers

In [11]:
def list_dedupe(x):
  """
  Convert list to dict and back to list to dedupe

  Parameters
  ----------
  x: list
      Python list object

  Returns
  -------
  dictionary:
    dictionary object with duplicated removed
  """
  return list(dict.fromkeys(x))
